In [1]:
#| include: false

import os
import pandas as pd
import numpy as np
# import plotly.express as px
import toml
# import psrc_theme
from pathlib import Path
import summary_data

# to show plotly figures in quarto HTML file
# import plotly.io as pio
# pio.renderers.default = "plotly_mimetype+notebook_connected"
# pio.templates.default = "simple_white+psrc_color" # set plotly template

config = summary_data.CONFIG
all_runs = summary_data.ALL_RUNS

In [2]:
# all_runs

- run locations

In [3]:
pd.DataFrame.from_dict(all_runs, orient='index',
                       columns=['run location'])

,run location
current run,\\modelstation2\c$\Workspace\sc_2023_04_23_25


## System Summary

In [4]:
# network summary: 'VMT','VHT','total_delay'
df_network = summary_data.load_agg_data('network/network_results.csv')

In [5]:
# Get light rail boardings
df_boardings = summary_data.load_agg_data('transit/daily_boardings_by_agency.csv')

In [6]:
network_summary = df_network.groupby('source')[['VMT','VHT','total_delay']].sum()
network_summary = network_summary.rename(columns={'total_delay': 'Delay'})
# transit boardings
df_boardings = summary_data.load_agg_data('transit/daily_boardings_by_agency.csv')
transit_ridership = df_boardings.groupby('source')['boardings'].sum()
#Light rail boardings
df_line_boardings = summary_data.load_agg_data('transit/transit_line_results.csv')
lr_boardings = df_line_boardings[df_line_boardings['mode']=='r']
lr_ridership = lr_boardings.groupby('source')['boardings'].sum()
# mode share
df_trip = summary_data.load_agg_data('agg/dash/mode_share_county.csv')
df = df_trip.groupby('source')['trexpfac'].sum() # total trips
transit_share = df_trip.loc[df_trip['mode']=="Transit"].groupby('source')['trexpfac'].sum()/df

# emission
df_emissions = summary_data.load_agg_data('emissions/emissions_summary.csv')
df = df_emissions.loc[(df_emissions['veh_type'].isin(['light','medium','heavy'])) & \
                      (df_emissions['pollutant_name']=="CO2 Equivalent")].copy()
CO2e = df.groupby('source')['total_daily_tons'].sum()

In [7]:
run_summary2 = pd.DataFrame({
    'Transit Boardings': transit_ridership,
    'Light Rail Boardings': lr_ridership,
    '% Transit': transit_share,
    'CO2e': CO2e
})

pd.concat([network_summary,run_summary2], axis=1).style.\
        format('{:,.0f}', subset=['VMT','VHT','Delay','Transit Boardings','Light Rail Boardings','CO2e']).\
        format('{:.1%}', subset=['% Transit'])

,VMT,VHT,Delay,Transit Boardings,Light Rail Boardings,% Transit,CO2e
source,,,,,,,
current run,"82,708,168","2,526,821","221,324","505,909","94,212",2.2%,"41,517"
